In [18]:
import cobra

In [19]:
import pandas as pd

In [20]:
from cobra.io import load_model

In [21]:
from cobra.flux_analysis import flux_variability_analysis

In [22]:
model = load_model("iJO1366") #no need to import from matlab

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [12]:
df = pd.read_csv("OptForce_Ecoli_Suc_Comb10_20.csv")

In [16]:
ko_df = df[df["Type of regulation"] == "knockout"]

In [28]:
optList = list(set(list(ko_df["Force Set"])))

In [23]:
biomass = "BIOMASS_Ec_iJO1366_core_53p95M"

In [24]:
model.objective = biomass

In [25]:
maxG = model.optimize().objective_value

In [26]:
maxG

0.9823718127269787

In [29]:
initFluxList = []
for rec in optList:
    initFlux = model.reactions.get_by_id(rec).summary().to_frame().loc[rec, "flux"]
    initFluxList.append(initFlux)

In [30]:
initFluxList

[17.578933530254233,
 0.0,
 0.0,
 0.0,
 10.610425051825072,
 0.5165753058634199,
 0.0,
 0.3496064807132768,
 0.0,
 -1.6472304460617337e-14,
 0.4177860316319673,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.3496064807132768,
 0.0,
 0.0,
 5.918065146483437,
 0.3496064807132768,
 -2.8760693263127175,
 0.0,
 0.00023969872230538257,
 0.8653751925204736,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.9476263040826485,
 1.9589431790865617,
 0.3496064807132768,
 0.0,
 -10.610425051825072,
 0.0,
 -1.6472304460617337e-14,
 0.0,
 0.0,
 0.0,
 0.0,
 0.17771597278137405,
 0.3496064807132768,
 0.0,
 0.0,
 0.0,
 -3.441826094608491,
 0.0,
 0.0,
 2.5786896606512517,
 0.00043912020028926205,
 0.9476263040826485,
 1.9647436254539556e-06,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3496064807132768,
 0.0,
 -0.0006611362299652629,
 0.0,
 0.0,
 0.0,
 0.0,
 -17.578933530254236,
 0.0,
 0.0,
 0.0,
 0.01577787368420799,
 0.0,
 0.0,
 1.6472304460617337e-14,
 0.0,
 0.0,
 0.9476263040826485,
 0.0,
 0.0,
 0.3496064807132768]

In [31]:
init_df = pd.DataFrame()

In [32]:
init_df["rec"] = optList
init_df["initFlux"] = initFluxList

In [33]:
init_df

,rec,initFlux
0,O2tex,17.578934
1,ACPS1,0.000000
2,EX_hxa_e,0.000000
3,FA120ACPHi,0.000000
4,NH4tex,10.610425
...,...,...
74,LPLIPAL2ATE161,0.000000
75,PItex,0.947626
76,PPA,0.000000
77,EX_mnl_e,0.000000


In [34]:
model.reactions.get_by_id(biomass).lower_bound = maxG*0.5 #min to half of max growth rate

In [35]:
model.objective = 'EX_succ_e'

In [36]:
flux_variability_analysis(model, "EX_succ_e", loopless=True)

,minimum,maximum
EX_succ_e,8.56108,8.56108


In [37]:
optFluxList = []
for rec in optList:
    optFlux = model.reactions.get_by_id(rec).summary().to_frame().loc[rec, "flux"]
    optFluxList.append(optFlux)

In [38]:
init_df["optFlux"] = optFluxList

In [44]:
init_df.to_csv("ko_init_opt.csv")

In [46]:
init_df

,rec,initFlux,optFlux
0,O2tex,17.578934,8.825687
1,ACPS1,0.000000,0.000000
2,EX_hxa_e,0.000000,0.000000
3,FA120ACPHi,0.000000,0.000000
4,NH4tex,10.610425,5.305213
...,...,...,...
74,LPLIPAL2ATE161,0.000000,0.000000
75,PItex,0.947626,0.473813
76,PPA,0.000000,0.000000
77,EX_mnl_e,0.000000,0.000000


In [50]:
opt_df  = init_df[(init_df["initFlux"] != 0.0) & (init_df["optFlux"] != 0.0)]

In [55]:
KOlist = list(opt_df["rec"])

In [56]:
objList = []

In [57]:
for rec in KOlist:
    with model:
        model.reactions.get_by_id(rec).knock_out()
        objList.append(model.optimize().objective_value)

/home/chenjunyu/miniconda3/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver sta

In [58]:
objList

[0.1628104452668676,
 0.0,
 0.0,
 8.467377465289465,
 8.467103037451189,
 8.467377465289465,
 8.451120724756539,
 8.467377465289347,
 8.467377465289347,
 8.562076604762714,
 8.562076604762701,
 8.546907796792528,
 8.519500341208262,
 8.519500341208262,
 8.542106614939016,
 8.467377465289383,
 8.438594936068373,
 8.561079850467241,
 8.561079850467241,
 8.535787333454362,
 8.561079780547548,
 8.46737746528943,
 8.561079850467292,
 0.0,
 8.560364375266879,
 8.560364375266879,
 8.519500341208303]

In [59]:
len(objList)

27